In [ ]:
from matplotlib import cm
from matplotlib import pyplot
from numpy import linalg
from numpy import random
from random import sample
from scipy import signal 
from scipy import stats
from shapely.geometry import Point, Polygon
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_samples
from sklearn.metrics import silhouette_score
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict

import dmr
import datetime
import foursquare
import geopandas
import math
import numpy
import pandas
import pickle
import pymongo
import time

In [ ]:
tessellation_uri = 'http://dati.comune.milano.it/dataset/806829b9-134b-40cf-b0e0-03e66c4f76d7/resource/3e355dd1-a8b8-483d-ac4d-03a62232ef38/download/ace_maggio_2011.geojson'

# Lantent Activity in Mobility Patterns

In [ ]:
def extract_category(c):
    if not c['categories']:
        return [(c['id'],c['name'])]
    else:
        l = [(c['id'],c['name'])]
        for ca in c['categories']:
            l.extend(extract_category(ca))
        return l

In [ ]:
categories_name = ['Arte e intrattenimento',
 'Istituti superiori e università',
 'Eventi',
 'Cibi',
 'Posti professionali e altri',
 'Locali notturni',
 'All\'aperto & Ricreativi',
 'Negozi e servizi',
 'Viaggi e trasporti',
 'Residenza']

### Pattern di mobilità

In [ ]:
bikemi_dataframe = pandas.DataFrame()

In [ ]:
station_lookup = {}
fwd_lookup, bwd_lookup = {} , {}

In [ ]:
def update_bwd():
    global bwd_lookup
    bwd_lookup = { v:k for k,v in fwd_lookup.items()}

In [ ]:
columns_filter = ['Data_prelievo'
                  , 'Gio_settimana_prelievo'
                  , 'Festivo_feriale_prelievo'
                  , 'Stazione_prelievo'
                  , 'Durata_sec'
                  , 'Data_arrivo'
                  , 'Gio_settimana_arrivo'
                  , 'Festivo_feriale_arrivo'
                  , 'Stazione_arrivo'
                 ]

In [ ]:
#unione dei dataframe
# for yy in range(2015, 2019): 
#     for mm in range(1, 13):
#         try:
#             csv_uri = '/home/datasets/bikemi/{0}/{1:02d} {0}.csv'.format(yy,mm)
#             next_df = pandas.read_csv(csv_uri,
#                            lineterminator ='\r',
#                            encoding = 'iso8859_2',
#                            sep = ';',
#                            parse_dates = ['Data_prelievo','Data_arrivo'],
#                            date_parser = lambda x: datetime.datetime.strptime(x,'%d/%m/%y %H:%M'),
#                            decimal = '.'
#                           )
            
#         except FileNotFoundError:
#             pass
#         else:
#             for _, i in next_df.iterrows():
                
#                 # LOOKUP
#                 a = i['Stazione_prelievo']
#                 b = i['Stazione_arrivo']
                
#                 if a not in station_lookup:
#                     station_lookup[a] = {i['Nome_stazione_prelievo']}
#                 else:
#                     station_lookup[a].add(i['Nome_stazione_prelievo'])
                    
#                 if b not in station_lookup:
#                     station_lookup[b] = {i['Nome_stazione_arrivo']}
#                 else:
#                     station_lookup[b].add(i['Nome_stazione_arrivo'])
                
#             #SAVE
#             bikemi_dataframe = pandas.concat([bikemi_dataframe, next_df[columns_filter]])


# pickle.dump(bikemi_dataframe, open( "archive/bikemi_dataframe.pkl", "wb" ) )
# pickle.dump(station_lookup, open( "archive/station_lookup.pkl", "wb" ) )

In [ ]:
bikemi_dataframe = pickle.load(open('archive/bikemi_dataframe.pkl', 'rb'))

In [ ]:
tmp = pickle.load(open('archive/station_lookup.pkl', 'rb'))
station_lookup = {k:list(v)[0] for k,v in tmp.items()}

In [ ]:
gps = pickle.load(open('/home/datasets/bikemi/station_gps_location.pkl', 'rb'))

### Correzione stazioni 

In [ ]:
# stazioni inutilizzate
for i in list(gps):
    if i not in station_lookup:
        del gps[i]

In [ ]:
# stazioni mancanti
gps[2] = gps[402]
gps[903] = gps[263]

gps[90] = (45.484649, 9.195576)
gps[92] = (45.465589, 9.186123)

In [ ]:
# relookupping
new_gps = []
for i in range(max(gps)+1):
    if i in gps:
        fwd_lookup[i] = len(new_gps)
        new_gps.append(gps[i])
gps = new_gps
update_bwd()

### Correzione POIs 

In [ ]:
# 4square
CLIENT_ID = 'XDNRQIG15QP4PUZEMAGGVBPYHV1WXQMFXXAXZM410USDSTC3'
CLIENT_SECRET = 'M55BW0UXPLUGACETI5ENMSF3WXGLDIENZAHE5VXPPVBYLSWO'
client = foursquare.Foursquare(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
categories = client.venues.categories()

In [ ]:
# lookup macrocateg
category_lookup = {}

In [ ]:
for macro_category in categories['categories']:
    for id_cat, name_cat in extract_category(macro_category):
        category_lookup[id_cat] = (macro_category['id'],macro_category['name'],name_cat)

In [ ]:
# 4square DB
mongo_conn = pymongo.MongoClient("mongodb://marvin.nptlab.di.unimi.it")
six_db = mongo_conn['FoursquarePlacesMilan']
places = six_db['places']

In [ ]:
pois = []

In [ ]:
# pulizia POIS
for i in places.find():
        tmp = {
            'name':i['name'],
            'location':(
                float(i['location']['lat']['$numberDouble']),
                float(i['location']['lng']['$numberDouble'])
            ),
            'categories':{category_lookup[j['id']][0] for j in i['categories']}
        }
        pois.append(tmp)

### Tassellamento 

In [ ]:
raw_aces = geopandas.read_file(tessellation_uri)
vor = {}

In [ ]:
# poligoni delle regioni
for _, r in raw_aces.iterrows():
    lat = r.geometry.exterior.xy[1]
    lng = r.geometry.exterior.xy[0]
    vor[int(r.ACE)-1] = Polygon([(lat[i], lng[i]) for i in range(len(lat))]) # L'ARRAY ACE PARTE DA 1

vor = numpy.array([vor[i] for i in range(len(vor))])

In [ ]:
# relookupping
for g in range(len(gps)):
    for r in range(len(vor)):
        if vor[r].contains(Point(gps[g])):
            fwd_lookup[bwd_lookup[g]] = r
            
del bwd_lookup

In [ ]:
# regioni non vuote
unempty = numpy.array([True]*len(vor))
for r in range(len(vor)):
    unempty[r] = r in fwd_lookup.values()

vor = vor[unempty]

In [ ]:
# relookupping
for k,v in dict(zip((i for i, kept in enumerate(unempty) if kept), range(len(vor)))).items():
    for f,b in fwd_lookup.items():
        if b == k:
            fwd_lookup[f] = v

### TF-IDF e SVD

In [ ]:
# categorie: solo quelle usate, senza lista completa
categories = set()
for i in pois:
    for j in i['categories']: 
        categories.add(j)
categories = list(categories)
categories.sort()

In [ ]:
# frequenze assolute
# abs_frequencies = numpy.zeros([len(vor), len(categories)])
# for i in range(len(vor)):
#     for p in pois:
#         if vor[i].contains(Point(p['location'])):
#             for k in p['categories']:
#                 abs_frequencies[i,categories.index(k)] += 1

# pickle.dump(abs_frequencies, open( "archive/abs_frequencies.pkl", "wb" ) )

In [ ]:
abs_frequencies = pickle.load(open('archive/abs_frequencies.pkl', 'rb'))

In [ ]:
dim_cat = [sum([abs_frequencies[i,j] for i in range(len(vor))]) for j in range(len(categories))] # n° poi per categoria
dim_reg = [sum([abs_frequencies[i,j] for j in range(len(categories))]) for i in range(len(vor))] # n°  poi per regione
num_reg = [len([i for i in range(len(vor)) if abs_frequencies[i,j]]) for j in range(len(categories))] # n° regioni per categoria

In [ ]:
tf_idf = numpy.zeros([len(vor),len(categories)])

In [ ]:
for i in range(len(vor)):
    for j in range(len(categories)):
        tf = dim_cat[j]/dim_reg[i]
        idf = len(vor)/num_reg[j]
        tf_idf[i,j] = tf*math.log(idf)

In [ ]:
u, _, _ = linalg.svd(tf_idf, full_matrices=True)

### Cuboidi

In [ ]:
pickle.dump(fwd_lookup, open( "archive/fwd_lookup.pkl", "wb" ))

In [ ]:
time_gran = 10*60
time_bin = [i for i in range(0,60*60*24*2,time_gran)]

In [ ]:
def i_time_bin(t,f):
    numpy.floor((60*(60*(24*(f)+t.hour)+t.minute)+t.second)/time_gran)

In [ ]:
arriving_cuboid = numpy.zeros([len(vor), len(vor), len(time_bin)])
leaving_cuboid = numpy.zeros([len(vor), len(vor), len(time_bin)])

In [ ]:
# for _, r in bikemi_dataframe.iterrows():
#     src = fwd_lookup[r['Stazione_prelievo']]
#     dst = fwd_lookup[r['Stazione_arrivo']]
    
#     leav_time = i_time_bin(r['Data_prelievo'], r['Festivo_feriale_prelievo'])
#     arr_time = i_time_bin(r['Data_arrivo'], r['Festivo_feriale_arrivo'])
    
#     leaving_cuboid[src, dst, leav_time] += 1
#     arriving_cuboid[src, dst, arr_time] += 1

In [ ]:
arriving_cuboid = pickle.load(open('archive/arriving_cuboid.pkl', 'rb'))
leaving_cuboid = pickle.load(open('archive/leaving_cuboid.pkl', 'rb'))

In [ ]:
docs = {}

In [ ]:
for i in range(len(vor)):
    a = [arriving_cuboid[r,i,t] for r in range(len(vor)) for t in range(len(time_bin))]
    l = [leaving_cuboid[i,r,t] for r in range(len(vor)) for t in range(len(time_bin))]
    m = a + l
    v = u[i]+[1]
    docs[i] = {'v':v, 'm':m}

In [ ]:
vocab = set([docs[i]['m'][n] for i in range(len(vor)) for n in range(len(docs[i]['m']))])
vocab = list(vocab)
vocab.sort()

In [ ]:
tmp = []
for i in range(len(vor)):
    tmp += docs[i]['m']

In [ ]:
fig = pyplot.figure(figsize=(16.5, 6.7))
ax = fig.gca()
eps = 10
yell = 10
y, x = numpy.histogram(tmp, numpy.arange(0, max(tmp)+eps, eps))
ax.plot(x[yell:-1], y[yell:], marker='.',ms=2.5, linestyle='', color='maroon')

In [ ]:
# fig = pyplot.figure(figsize=(16.5, 6.7))
# ax = fig.gca()
# ecdf = ECDF(tmp)
# ax.plot(ecdf.x, ecdf.y, marker='.',ms=0.1, linestyle='--', color='coral')
# ax.set_xlim(0, 40)

In [ ]:
# numpy.mean(tmp), numpy.percentile(tmp, 50), numpy.std(tmp)

###  Probablilità congiunta

In [ ]:
probs = numpy.array([[abs_frequencies[r,k]/sum(abs_frequencies[r]) for k in range(len(categories))] for r in range(len(vor))])

for k in range(len(categories)):
    tmp = probs[:,k]
    print((numpy.mean(tmp), numpy.std(tmp)))

In [ ]:
# move_probs = numpy.zeros([len(categories),len(categories),len(vor),len(vor)])

# for i, r in bikemi_dataframe.iterrows():
#     src = fwd_lookup[r['Stazione_prelievo']]
#     dst = fwd_lookup[r['Stazione_arrivo']]
#     for ks in range(len(categories)):
#         for kd in range(len(categories)):
#             move_probs[ks,kd,src,dst] += probs[src,ks]*probs[dst,kd]

### Diriclet Multinomial Regression di Kato 

In [ ]:
corpus, vecs = [], []

In [ ]:
# params
K = 10
sigma = 1.0
beta = 0.01

In [ ]:
for r in range(len(docs)):
    corpus.append(numpy.array(docs[r]['m']))
    vecs.append(numpy.array(docs[r]['v']))
corpus, vecs = numpy.array(corpus), numpy.array(vecs)

In [ ]:
# learning
voca = dmr.Vocabulary()
docs = voca.read_corpus(corpus)

lda = dmr.DMR(K, sigma, beta, docs, vecs, voca.size())

lda.learning(iteration=150, voca=voca)

In [ ]:
# # topic probability of each document
# tdist = lda.topicdist()
# extracted = []
# for doc, vec, td in zip(corpus, vecs, tdist):
#     print("For: ", doc, "Max topic: ", np.argmax(td), "Max prob.: ", np.max(td))
#     #print("ALPHA", np.dot(vec, lda.Lambda.T))
#     extracted.append(np.argmax(td))

In [ ]:
tmp = lda.z_m_n
t = [ tmp[j][i] for j in range(len(tmp)) for i in range(len(tmp[j])) ]

In [ ]:
fig = pyplot.figure(figsize=(16.5, 6.7))
ax = fig.gca()
y, x = numpy.histogram(t, numpy.arange(0, max(t)+2, 1))
ax.plot(x[:-1], y, marker='.',ms=11.0, linestyle='', color='purple')

In [ ]:
reg_a = 1 # Duomo
reg_b = 3 # Cadorna
arr_part_a = 0
hh, mm = 12, 10

categories_name[tmp[fwd_lookup[reg_a]][int((60*hh+mm)/10)*reg_b+(arr_part_a*14400)]]